In [48]:
import numpy as np 
import pandas as pd 
import os
import re
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import tensorflow as tf

In [2]:
train=pd.read_json("C:\\Users\\Aayush Kandpal\\Downloads\\archive (1)\\Sarcasm_Headlines_Dataset_v2.json",lines=True)
test=pd.read_json("C:\\Users\\Aayush Kandpal\\Downloads\\archive (1)\\Sarcasm_Headlines_Dataset.json",lines=True)

In [3]:
train.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [4]:
test.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [5]:
train=train.drop('article_link',axis=1)
test=test.drop('article_link',axis=1)

In [6]:
train.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


In [7]:
test.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [8]:
def clean_text(text):
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)        
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text) 
    text = re.sub(r"\'ll", " will", text)  
    text = re.sub(r"\'ve", " have", text)  
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"did't", "did not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"have't", "have not", text)
    text = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-]", "", text)
    return text

In [20]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def CleanTokenize(df):
    head_lines = list()
    lines = df["headline"].values.tolist()

    for line in lines:
        line = clean_text(line)
        head_lines.append(line)
    return head_lines

head_lines = CleanTokenize(train)
head_lines[0:20]

['thirtysomething scientists unveil doomsday clock of hair loss',
 'dem rep totally nails why congress is falling short on gender racial equality',
 'eat your veggies 9 deliciously different recipes',
 'inclement weather prevents liar from getting to work',
 'mother comes pretty close to using word streaming correctly',
 'my white inheritance',
 '5 ways to file your taxes with less stress',
 'richard bransons globalwarming donation nearly as much as cost of failed balloon trips',
 'shadow government getting too large to meet in marriott conference room b',
 'lots of parents know this scenario',
 'this lesbian is considered a father in indiana and an amazing one at that',
 'amanda peet told her daughter sex is a special hug',
 'what to know regarding current treatments for ebola',
 'chris christie suggests hillary clinton was to blame for boko harams kidnapping of hundreds of schoolgirls',
 'ford develops new suv that runs purely on gasoline',
 'uber ceo travis kalanick stepping down fr

In [24]:
import nltk
from nltk.corpus import stopwords
head_lines=[word for word in head_lines if not word in set(stopwords.words('english'))]

In [25]:
head_lines[0:5]

['thirtysomething scientists unveil doomsday clock of hair loss',
 'dem rep totally nails why congress is falling short on gender racial equality',
 'eat your veggies 9 deliciously different recipes',
 'inclement weather prevents liar from getting to work',
 'mother comes pretty close to using word streaming correctly']

In [26]:
y_train=train.is_sarcastic

In [28]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
head_lines=[ps.stem(word) for word in head_lines if not word in set(stopwords.words('english'))]

In [30]:
head_lines[0:5]

['thirtysomething scientists unveil doomsday clock of hair loss',
 'dem rep totally nails why congress is falling short on gender racial equ',
 'eat your veggies 9 deliciously different recip',
 'inclement weather prevents liar from getting to work',
 'mother comes pretty close to using word streaming correctli']

In [33]:
head_lines_test=CleanTokenize(test)
head_lines_test=[word for word in head_lines_test if not word in set(stopwords.words('english'))]
head_lines_test=[ps.stem(word) for word in head_lines_test if not word in set(stopwords.words('english'))]

In [50]:
vocab_size = 10000
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
embedding_dim=100

In [36]:
training_sentences=head_lines
testing_sentences=head_lines_test
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [38]:
y_test=test.is_sarcastic

In [59]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    
    tf.keras.layers.GlobalAveragePooling1D(),
    
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [60]:
model.fit(training_padded,y_train,epochs=30,verbose=2)

Epoch 1/30
895/895 - 11s - loss: 0.5240 - accuracy: 0.7349
Epoch 2/30
895/895 - 11s - loss: 0.3136 - accuracy: 0.8709
Epoch 3/30
895/895 - 11s - loss: 0.2540 - accuracy: 0.8969
Epoch 4/30
895/895 - 11s - loss: 0.2181 - accuracy: 0.9138
Epoch 5/30
895/895 - 11s - loss: 0.1954 - accuracy: 0.9230
Epoch 6/30
895/895 - 11s - loss: 0.1780 - accuracy: 0.9297
Epoch 7/30
895/895 - 11s - loss: 0.1608 - accuracy: 0.9392
Epoch 8/30
895/895 - 11s - loss: 0.1515 - accuracy: 0.9426
Epoch 9/30
895/895 - 11s - loss: 0.1429 - accuracy: 0.9474
Epoch 10/30
895/895 - 11s - loss: 0.1355 - accuracy: 0.9481
Epoch 11/30
895/895 - 11s - loss: 0.1271 - accuracy: 0.9543
Epoch 12/30
895/895 - 11s - loss: 0.1219 - accuracy: 0.9556
Epoch 13/30
895/895 - 11s - loss: 0.1145 - accuracy: 0.9588
Epoch 14/30
895/895 - 11s - loss: 0.1087 - accuracy: 0.9619
Epoch 15/30
895/895 - 11s - loss: 0.1047 - accuracy: 0.9638
Epoch 16/30
895/895 - 11s - loss: 0.1030 - accuracy: 0.9638
Epoch 17/30
895/895 - 11s - loss: 0.0970 - accura

In [61]:
pred=model.predict(testing_padded)


In [63]:
pred

array([[8.9184344e-03],
       [1.6447902e-04],
       [1.0000000e+00],
       ...,
       [2.9406071e-02],
       [8.3326995e-03],
       [3.9908290e-04]], dtype=float32)

In [64]:
pred1=pred>0.5
pred1

array([[False],
       [False],
       [ True],
       ...,
       [False],
       [False],
       [False]])

In [65]:
from sklearn.metrics import accuracy_score
print('Accuracy using our sarcasm predictor',accuracy_score(y_test,pred1))
from sklearn.metrics import classification_report
print(classification_report(y_test,pred1))

Accuracy using our sarcasm predictor 0.9906772997865888
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     14985
           1       0.99      0.99      0.99     11724

    accuracy                           0.99     26709
   macro avg       0.99      0.99      0.99     26709
weighted avg       0.99      0.99      0.99     26709

